In [ ]:
!pip install ultralytics > /dev/null

In [ ]:
import numpy as np
from ultralytics import YOLO

In [ ]:
!pip install moviepy > /dev/null

In [ ]:
!pip install pyannote.audio > /dev/null
!pip freeze | grep -i pyannote >/dev/null

In [ ]:
!pip install ffmpeg-python > /dev/null

In [ ]:
!unzip /content/ideal_poses_2.zip > /dev/null

In [ ]:
from models.settings.settings import VideoInfo, Settings
from models.settings.detect_xy import DetectXY

from processors.score_collector import VideoScoreCollector, AudioScoreCollector
from processors.video_sim_processor import VideoSimilarityProcessor
from processors.audio_sim_processor import AudioSimilarityProcessor
from processors.score_processor import ScoreProcessor
from processors.results.output_video_processor import OutputVideoProcessor
from processors.results.result_processor import ResultProcessor
from utils.utils import get_ideal_pose_list

from config import *

In [ ]:
model = YOLO(YOLO_MODEL)

In [ ]:
ideal_pose_list = get_ideal_pose_list(poses=POSES, ideal_pose_dir="/content/ideal_poses_2", yolo_model = model)

In [ ]:
video_info = VideoInfo.load(VIDEO_PATH)
settings = Settings(video_info=video_info,
                    base_path=BASE_PATH,
                    detect_area = DetectXY(min = np.array(DETECT_AREA["min"]), max = np.array(DETECT_AREA["max"])), # 指定しない場合消す
                    mask_area = DetectXY(min = np.array(MASK_AREA["min"]), max = np.array(MASK_AREA["max"]))
)

# processor
video_score_collector = VideoScoreCollector(settings=settings)
video_sim_processor = VideoSimilarityProcessor(
    model=model,
    ideal_pose_list=ideal_pose_list,
    video_score_collector=video_score_collector,
    settings=settings
)

audio_score_collector = AudioScoreCollector()
audio_similarity_processor = AudioSimilarityProcessor(
    audio_score_collector,
    settings
)

print("- 1: video similarity processor")
video_sim_processor.run()

print("- 2: audio similarity processor")
audio_similarity_processor.run()

print("- 3: score culcrator")
score_processor = ScoreProcessor(
    video_score_collector,
    audio_score_collector,
    settings.base_path)
score_processor.calculate()
video_score_list, audio_score_list = score_processor.get_scores()

output_video_processor = OutputVideoProcessor(settings,
                                              output_file_name="final_video.mp4")

result_processor = ResultProcessor(
    score_processor,
    output_video_processor,
    video_score_list,
    audio_score_list,
    settings
)

print("- 4: write video and csv")
tracked_people_xys, tracked_person_keypoints_list = video_sim_processor.extract_tracked_person_info()

result_processor.draw_output_video(tracked_people_xys, tracked_person_keypoints_list)
result_processor.write_csv()